# Pricing d'options exotiques par schéma de Crank–Nicolson (Black–Scholes PDE)

Dans ce notebook, on résout l'équation de Black–Scholes par un schéma implicite
**Crank–Nicolson** en coordonnées log(S) pour pricer :

- des options **européennes**,
- des options **américaines** (exercice continu),
- des options **bermudéennes** (exercice discret),
- des options **barrières** (up-and-out, double knock-out, etc.).

Nous travaillerons avec la variable :
\\[
x = \ln\left(\frac{S}{S_0}\right)
\\]
et le temps \( t \in [0, T] \), et nous discrétisons l'espace et le temps pour
résoudre numériquement la PDE.


In [1]:
import numpy as np
from scipy.sparse import diags
import matplotlib.pyplot as plt

%matplotlib inline


## PDE de Black–Scholes en log-prix

Sous Black–Scholes, le prix d’une option \( V(t,S) \) vérifie :

\\[
\frac{\partial V}{\partial t}
+ \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2}
+ (r - d) S \frac{\partial V}{\partial S}
- r V = 0.
\\]

On pose \( x = \ln(S/S_0) \), alors \( S = S_0 e^x \), et la PDE se réécrit
en termes de \( V(t,x) \). On obtient une équation de la forme :

\\[
\frac{\partial V}{\partial t}
= a(x) \frac{\partial^2 V}{\partial x^2}
+ b(x) \frac{\partial V}{\partial x}
+ c V.
\\]

Nous allons discrétiser \( x \) et \( t \) et utiliser un schéma de Crank–Nicolson :

\\[
A V^{n} = B V^{n+1},
\\]

où \( V^n \) est le vecteur des valeurs à l’instant \( t_n \).


In [2]:
class CrankNicolsonBS:
    """
    Solveur Crank–Nicolson pour la PDE de Black–Scholes en log(S).

    Typeflag:
        'Eu'  : option européenne
        'Am'  : option américaine (exercice possible à chaque date de grille)
        'Bmd' : option bermudéenne (exercice possible à certaines dates)
    cpflag:
        'c' : call
        'p' : put
    """

    def __init__(self, Typeflag, cpflag, S0, K, T, vol, r, d):
        self.Typeflag = Typeflag
        self.cpflag = cpflag
        self.S0 = S0
        self.K = K
        self.T = T
        self.vol = vol
        self.r = r
        self.d = d

    def CN_option_info(self,
                       Typeflag=None, cpflag=None,
                       S0=None, K=None, T=None, vol=None, r=None, d=None):
        """
        Méthode principale : résout la PDE et retourne (Price, Delta, Gamma, Theta).

        On utilise :
        - grille en x : [-x_max, x_max], où x_max ~ 5 * sigma * sqrt(T)
        - grille en t : J pas de temps
        - discrétisation en log(S) pour couvrir un range large de prix.
        """

        # --------- Mise à jour des paramètres éventuels ---------
        Typeflag = Typeflag or self.Typeflag
        cpflag   = cpflag   or self.cpflag
        S0       = S0       or self.S0
        K        = K        or self.K
        T        = T        or self.T
        vol      = vol      or self.vol
        r        = r        or self.r
        d        = d        or self.d

        # --------- Paramètres de la grille en x,t ---------
        mu    = (r - d - 0.5 * vol * vol)    # coefficient de drift en log(S)
        x_max = vol * np.sqrt(T) * 5         # intervalle en x large
        N     = 500                          # nb de pas en x
        dx    = 2 * x_max / N
        X     = np.linspace(-x_max, x_max, N + 1)  # points x
        n     = np.arange(0, N + 1)

        J  = 600        # nb de pas de temps
        dt = T / J

        # --------- Coefficients Crank–Nicolson ---------
        # Forme standard : a * V_{i+1} + b * V_i + c * V_{i-1}
        a = 0.25 * dt * ((vol**2) * (n**2) - mu * n)
        b = -0.5 * dt * ((vol**2) * (n**2) + r)
        c = 0.25 * dt * ((vol**2) * (n**2) + mu * n)

        # Matrices tri-diagonales A et B
        main_diag_A = 1 - b - 2*a
        upper_A     = a + c
        lower_A     = a - c

        main_diag_B = 1 + b + 2*a
        upper_B     = -a - c
        lower_B     = -a + c

        # Construction dense (pédagogique, pas optimal)
        A = np.zeros((N+1, N+1))
        B = np.zeros((N+1, N+1))

        np.fill_diagonal(A, main_diag_A)
        np.fill_diagonal(A[1:], lower_A[:-1])
        np.fill_diagonal(A[:,1:], upper_A[:-1])

        np.fill_diagonal(B, main_diag_B)
        np.fill_diagonal(B[1:], lower_B[:-1])
        np.fill_diagonal(B[:,1:], upper_B[:-1])

        Ainv = np.linalg.inv(A)  # numériquement, on préférerait un solveur tri-diagonal

        # --------- Payoff terminal à maturité ---------
        if cpflag == 'c':
            V = np.clip(S0 * np.exp(X) - K, 0, 1e10)
        elif cpflag == 'p':
            V = np.clip(K - S0 * np.exp(X), 0, 1e10)
        else:
            raise ValueError("cpflag doit être 'c' ou 'p'.")

        # V0 : payoff utilisé pour l'exercice anticipé (Am/Bmd)
        V0 = V.copy()

        # V1 : stocker V à l'avant-dernier pas de temps pour calculer Theta
        V1 = V.copy()

        # --------- Backward en temps, suivant le type d'option ---------
        if Typeflag == 'Am':
            # Option américaine : exercice possible à TOUTES les dates
            for j in range(J):
                if j == J - 1:
                    V1 = V.copy()

                # Schéma CN : V^{n} = A^{-1} B V^{n+1}
                V = B.dot(V)
                V = Ainv.dot(V)

                # Exercice anticipé : max(V, payoff)
                V = np.where(V > V0, V, V0)

        elif Typeflag == 'Bmd':
            # Option bermudéenne : exercice possible à certaines dates
            # Exemple pédagogique : exercice tous les 10 pas de temps
            exercise_step = 10

            for j in range(J):
                if j == J - 1:
                    V1 = V.copy()

                V = B.dot(V)
                V = Ainv.dot(V)

                # Exercice seulement à certains pas
                if j % exercise_step == 0:
                    V = np.where(V > V0, V, V0)

        elif Typeflag == 'Eu':
            # Option européenne : pas d'exercice anticipé
            for j in range(J):
                if j == J - 1:
                    V1 = V.copy()

                V = B.dot(V)
                V = Ainv.dot(V)

                # Conditions de bord (exemple pour un call)
                if cpflag == 'c':
                    V[0] = 0.0
                    V[-1] = S0 * np.exp(x_max) - K * np.exp(-r * dt * j)
                else:
                    # put : valeur proche de K e^{-r(T-t)} si S très petit
                    V[0]  = K * np.exp(-r * dt * (J - j))
                    V[-1] = 0.0
        else:
            raise ValueError("Typeflag doit être 'Eu', 'Am' ou 'Bmd'.")

        # --------- Extraction du prix et des grecs à S = S0 ---------
        # S0 correspond à x = 0, donc à l'indice n_mid ~ N/2
        n_mid = N // 2

        price = V[n_mid]

        # Delta : dérivée en S approximée par diff finie centrée
        Sp = S0 * np.exp(dx)
        Sm = S0 * np.exp(-dx)
        delta = (V[n_mid+1] - V[n_mid-1]) / (Sp - Sm)

        # Gamma : dérivée seconde en S, double différence finie
        dVdSp = (V[n_mid+1] - V[n_mid]) / (Sp - S0)
        dVdSm = (V[n_mid]   - V[n_mid-1]) / (S0 - Sm)
        gamma = (dVdSp - dVdSm) / ((Sp - Sm) / 2.0)

        # Theta : dérivée en temps approximée avec V1 (une étape de dt plus tard)
        theta = -(V[n_mid] - V1[n_mid]) / dt

        return price, delta, gamma, theta


In [3]:
# Bermuda Call
Bmd_call = CrankNicolsonBS('Bmd', 'c', S0=100, K=100, T=1.0,
                           vol=0.4, r=0.025, d=0.0175)

price, delta, gamma, theta = Bmd_call.CN_option_info()

print("Bermuda Call")
print("Price:", price)
print("Delta:", delta)
print("Gamma:", gamma)
print("Theta:", theta)


Bermuda Call
Price: nan
Delta: nan
Gamma: nan
Theta: nan


/tmp/ipykernel_69258/2731176041.py:124: RuntimeWarning: overflow encountered in dot
  V = Ainv.dot(V)
/tmp/ipykernel_69258/2731176041.py:124: RuntimeWarning: invalid value encountered in dot
  V = Ainv.dot(V)
/tmp/ipykernel_69258/2731176041.py:123: RuntimeWarning: overflow encountered in dot
  V = B.dot(V)
/tmp/ipykernel_69258/2731176041.py:123: RuntimeWarning: invalid value encountered in dot
  V = B.dot(V)


In [4]:
# Bermuda Put
Bmd_put = CrankNicolsonBS('Bmd', 'p', S0=100, K=100, T=1.0,
                          vol=0.4, r=0.025, d=0.0175)

price, delta, gamma, theta = Bmd_put.CN_option_info()

print("Bermuda Put")
print("Price:", price)
print("Delta:", delta)
print("Gamma:", gamma)
print("Theta:", theta)


/tmp/ipykernel_69258/2731176041.py:124: RuntimeWarning: invalid value encountered in dot
  V = Ainv.dot(V)


Bermuda Put
Price: nan
Delta: nan
Gamma: nan
Theta: nan


## Options barrières par Crank–Nicolson

On adapte maintenant le schéma PDE pour gérer des **barrières**.

Idée :

- On résout la même PDE, mais
- à chaque pas de temps on impose que la valeur soit **0** si le sous-jacent a "touché" une barrière de type knock-out.
- Exemple :
  - Up-and-out put : si \( S_t \ge H_u \), valeur = 0.
  - Double knock-out : si \( S_t \le H_d \) ou \( S_t \ge H_u \), valeur = 0.


In [5]:
def CN_Barrier_option(Typeflag, cpflag,
                      S0, K, Hu, Hd, T, vol, r, d):
    """
    Pricing d'une option barrière par Crank–Nicolson.

    Typeflag:
        'UNO' : Up-and-out (knock-out par le haut)
        'DNO' : Double knock-out (Hd < S < Hu)
    cpflag:
        'c' : call
        'p' : put

    Hu : barrière haute
    Hd : barrière basse (peut être 0 si pas de barrière basse effective)
    """

    mu    = (r - d - 0.5 * vol * vol)
    x_max = vol * np.sqrt(T) * 5
    N     = 500
    dx    = 2 * x_max / N
    X     = np.linspace(-x_max, x_max, N + 1)
    n     = np.arange(0, N + 1)

    J  = 600
    dt = T / J

    # Coeffs Crank–Nicolson
    a = 0.25 * dt * ((vol**2) * (n**2) - mu * n)
    b = -0.5 * dt * ((vol**2) * (n**2) + r)
    c = 0.25 * dt * ((vol**2) * (n**2) + mu * n)

    main_diag_A = 1 - b - 2*a
    upper_A     = a + c
    lower_A     = a - c

    main_diag_B = 1 + b + 2*a
    upper_B     = -a - c
    lower_B     = -a + c

    A = np.zeros((N+1, N+1))
    B = np.zeros((N+1, N+1))

    np.fill_diagonal(A, main_diag_A)
    np.fill_diagonal(A[1:], lower_A[:-1])
    np.fill_diagonal(A[:,1:], upper_A[:-1])

    np.fill_diagonal(B, main_diag_B)
    np.fill_diagonal(B[1:], lower_B[:-1])
    np.fill_diagonal(B[:,1:], upper_B[:-1])

    Ainv = np.linalg.inv(A)

    # Payoff terminal
    S_grid = S0 * np.exp(X)
    if cpflag == 'c':
        V = np.clip(S_grid - K, 0, 1e10)
    elif cpflag == 'p':
        V = np.clip(K - S_grid, 0, 1e10)
    else:
        raise ValueError("cpflag doit être 'c' ou 'p'.")

    # Condition de barrière à maturité (knock-out) :
    if Typeflag == 'UNO':
        # Up-and-out : si S_T >= Hu, payoff=0
        V = np.where(S_grid < Hu, V, 0.0)
    elif Typeflag == 'DNO':
        # Double knock-out : si S_T <= Hd ou S_T >= Hu, payoff=0
        V = np.where((S_grid > Hd) & (S_grid < Hu), V, 0.0)
    else:
        raise ValueError("Typeflag doit être 'UNO' ou 'DNO' pour les barrières.")

    V1 = V.copy()

    # Backward en temps
    for j in range(J):
        if j == J - 1:
            V1 = V.copy()

        V = B.dot(V)
        V = Ainv.dot(V)

        # Appliquer la condition de barrière à chaque pas
        S_grid = S0 * np.exp(X)
        if Typeflag == 'UNO':
            V = np.where(S_grid < Hu, V, 0.0)
        elif Typeflag == 'DNO':
            V = np.where((S_grid > Hd) & (S_grid < Hu), V, 0.0)

    # Extraction du prix et des grecs à S=S0
    n_mid = N // 2

    price = V[n_mid]

    Sp = S0 * np.exp(dx)
    Sm = S0 * np.exp(-dx)

    delta = (V[n_mid+1] - V[n_mid-1]) / (Sp - Sm)

    dVdSp = (V[n_mid+1] - V[n_mid]) / (Sp - S0)
    dVdSm = (V[n_mid]   - V[n_mid-1]) / (S0 - Sm)
    gamma = (dVdSp - dVdSm) / ((Sp - Sm) / 2.0)

    theta = -(V[n_mid] - V1[n_mid]) / dt

    return price, delta, gamma, theta


In [6]:
# Up-and-out Put
price, delta, gamma, theta = CN_Barrier_option(
    Typeflag='UNO',
    cpflag='p',
    S0=100,
    K=90,
    Hu=120,  # barrière haute
    Hd=0.0,  # pas de barrière basse effective
    T=1.0,
    vol=0.5,
    r=0.025,
    d=0.0175
)

print("Up-and-out Put")
print("Price:", price)
print("Delta:", delta)
print("Gamma:", gamma)
print("Theta:", theta)


/tmp/ipykernel_69258/640426439.py:80: RuntimeWarning: invalid value encountered in dot
  V = Ainv.dot(V)


Up-and-out Put
Price: nan
Delta: nan
Gamma: nan
Theta: nan


In [7]:
# Double knock-out Put
price, delta, gamma, theta = CN_Barrier_option(
    Typeflag='DNO',
    cpflag='p',
    S0=100,
    K=90,
    Hu=120,  # barrière haute
    Hd=80,   # barrière basse
    T=1.0,
    vol=0.5,
    r=0.025,
    d=0.0175
)

print("Double knock-out Put")
print("Price:", price)
print("Delta:", delta)
print("Gamma:", gamma)
print("Theta:", theta)


/tmp/ipykernel_69258/640426439.py:80: RuntimeWarning: invalid value encountered in dot
  V = Ainv.dot(V)


Double knock-out Put
Price: nan
Delta: nan
Gamma: nan
Theta: nan
